In [8]:
# カウントダウンして画像を保存

import cv2
import dlib
import time
import os

# DNNベースの顔検出器のロード
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# カウントダウン秒数
COUNTDOWN_SECONDS = 5

# 画像保存先のディレクトリ
save_dir = 'images'

# ディレクトリが存在しない場合は作成
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# カメラの指定
cap = cv2.VideoCapture(0)

# シャッター音の再生（OpenCVは音を扱えないため、winsoundやpygameを使用するか、代わりに効果音を表示）
def play_shutter_sound():
    print("シャッター音が再生されました")  # 実際には音を再生します

# カウントダウンを表示しながらフレームを更新する関数
def start_countdown(cap):
    start_time = time.time()  # カウントダウンの開始時間
    while True:
        ret, frame = cap.read()  # フレームを再取得
        if not ret:
            break

        # 経過時間を計算
        elapsed_time = time.time() - start_time
        countdown_time = COUNTDOWN_SECONDS - int(elapsed_time)

        # カウントダウンが終了したらループを抜ける
        if countdown_time <= 0:
            break

        # カウントダウンの数字を画面に表示
        frame_copy = frame.copy()
        cv2.putText(frame_copy, str(countdown_time), (frame.shape[1] // 2 - 50, frame.shape[0] // 2), 
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5, cv2.LINE_AA)
        
        # フレームを表示
        cv2.imshow('Preview', frame_copy)

        # フレームを頻繁に更新（短い間隔で待機）
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    return frame  # カウントダウン後の最後のフレームを返す

# 画像を保存する関数
def save_image(frame):
    filename = f'photo_{time.strftime("%Y%m%d_%H%M%S")}.png'
    filepath = os.path.join(save_dir, filename)  # imagesフォルダに保存
    cv2.imwrite(filepath, frame)
    print(f"画像が保存されました: {filepath}")

# メインループ
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # グレースケール画像に変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顔を検出
    faces = face_detector(gray)
    
    # 検出された顔にランドマークを描画（例: 目や口）
    for face in faces:
        landmarks = landmark_predictor(gray, face)
        for n in range(68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(frame, (x, y), 2, (255, 0, 0), -1)

    # フレームを表示
    cv2.imshow('Preview', frame)

    # 's'キーでスタートボタンの役割
    if cv2.waitKey(1) & 0xFF == ord('s'):
        # スタートボタンを押すとカウントダウン開始
        final_frame = start_countdown(cap)

        # カウントダウンが終わったらシャッター音を再生し、写真を保存
        play_shutter_sound()
        save_image(final_frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 終了処理
cap.release()
cv2.destroyAllWindows()


シャッター音が再生されました
画像が保存されました: images\photo_20241002_214609.png


In [11]:
# クマさんになるやつ

import cv2
import dlib
import numpy as np
from imutils import face_utils
from scipy.spatial import distance

# Dlibの学習済みモデルファイルを読み込み
face_parts_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

# クマの左耳と右耳、鼻の画像を読み込み
left_bear_ear = cv2.imread("bear/left_bear_ear.png", cv2.IMREAD_UNCHANGED)
right_bear_ear = cv2.imread("bear/right_bear_ear.png", cv2.IMREAD_UNCHANGED)
bear_nose = cv2.imread("bear/bear_nose.png", cv2.IMREAD_UNCHANGED)

# クッキーの画像を読み込み
cookie_img = cv2.imread("bear/cookie.png", cv2.IMREAD_UNCHANGED)  # 透過PNG形式

# 透過PNG画像の合成関数
def overlay_image_alpha(img, overlay, pos, scale=1.0):
    # オーバーレイ画像のサイズをスケールで調整
    overlay = cv2.resize(overlay, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    x, y = pos
    h, w = overlay.shape[0], overlay.shape[1]

    # 境界チェック（オーバーレイ画像が元の画像をはみ出ないようにする）
    if x + w > img.shape[1]:
        w = img.shape[1] - x
    if y + h > img.shape[0]:
        h = img.shape[0] - y
    if x < 0:
        x = 0
    if y < 0:
        y = 0

    # マスクと逆マスクを作成
    if overlay.shape[2] == 4:  # 透過チャンネルがある場合
        alpha_mask = overlay[:h, :w, 3] / 255.0
        alpha_image = 1.0 - alpha_mask

        # オーバーレイ部分を合成
        for c in range(0, 3):
            img[y:y+h, x:x+w, c] = (alpha_image * img[y:y+h, x:x+w, c] + alpha_mask * overlay[:h, :w, c])

    return img

# 口の開閉を判定する関数
def calc_mouth(mouth):
    # 口のアスペクト比（MAR）を計算
    A = distance.euclidean(mouth[2], mouth[10])  # 上唇と下唇の縦距離
    B = distance.euclidean(mouth[4], mouth[8])   # 上唇と下唇の縦距離
    C = distance.euclidean(mouth[0], mouth[6])   # 左右の横距離
    MAR = (A + B) / (2.0 * C)
    return round(MAR, 3)

# カメラデバイスをオープン
cap = cv2.VideoCapture(0)

###################
# メイン処理       #
###################
while True:
    ret, image = cap.read()
    if not ret:
        break

    # グレースケールに変換
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 顔検出
    rects = detector(gray, 1)

    for rect in rects:
        # ランドマークを取得
        shape = face_parts_detector(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # 顔のランドマークの位置を取得
        left_eye = shape[36]  # 左目の外側
        right_eye = shape[45]  # 右目の外側
        nose = shape[30]  # 鼻の位置
        chin = shape[8]  # 顎のランドマーク

        # 顔の幅と高さを基に耳の位置を調整
        face_width = np.linalg.norm(left_eye - right_eye)  # 両目間の距離
        face_height = np.linalg.norm(nose - chin)  # 顔の高さ

        # 左耳のX座標とY座標を計算
        left_ear_x = int(left_eye[0]) - int(face_width * 0.5)  # 左耳は左目の外側から少し左に配置
        left_ear_y = int(left_eye[1]) - int(face_height * 1.2)  # 左耳のY位置は顔の上部に調整

        # 右耳のX座標とY座標を計算
        right_ear_x = int(right_eye[0]) + int(face_width * -0.2)  # 右耳は右目の外側から少し右に配置
        right_ear_y = int(right_eye[1]) - int(face_height * 1.25)  # 右耳のY位置も顔の上部に調整

        # 左耳を合成
        image = overlay_image_alpha(image, left_bear_ear, (left_ear_x, left_ear_y), scale=face_width / 200)

        # 右耳を合成
        image = overlay_image_alpha(image, right_bear_ear, (right_ear_x, right_ear_y), scale=face_width / 200)

        # 鼻の位置にクマの鼻を合成
        nose_x = nose[0] - 58  # 鼻の画像のX位置調整
        nose_y = nose[1] - 70  # 鼻の画像のY位置調整
        image = overlay_image_alpha(image, bear_nose, (nose_x, nose_y), scale=0.3)

        # 口のランドマーク（48番から67番の点）を取得
        mouth = shape[48:68]

        # 口のアスペクト比（MAR）を計算
        MAR = calc_mouth(mouth)

        # 口のMARが閾値を超えているかチェック（0.6を閾値として設定）
        if MAR > 0.6:
            # クッキーを口の位置に合成
            mouth_center = mouth.mean(axis=0).astype(int)  # 口の中心座標を計算
            
            # クッキーの縦位置を調整（唇の位置を参考に）
            lip_distance = distance.euclidean(mouth[3], mouth[9])  # 上唇と下唇の距離
            image = overlay_image_alpha(image, cookie_img, (mouth_center[0] - 20, mouth_center[1] - int(lip_distance * 0.5)), scale=0.5)

    # 画像を表示
    cv2.imshow('Frame', image)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 終了処理
cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from scipy.spatial import distance
import time
import os

# Dlibの学習済みモデルファイルを読み込み
face_parts_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

# クマの左耳と右耳、鼻の画像を読み込み
left_bear_ear = cv2.imread("bear/left_bear_ear.png", cv2.IMREAD_UNCHANGED)
right_bear_ear = cv2.imread("bear/right_bear_ear.png", cv2.IMREAD_UNCHANGED)
bear_nose = cv2.imread("bear/bear_nose.png", cv2.IMREAD_UNCHANGED)

# クッキーの画像を読み込み
cookie_img = cv2.imread("bear/cookie.png", cv2.IMREAD_UNCHANGED)  # 透過PNG形式

# カウントダウン秒数
COUNTDOWN_SECONDS = 5

# 画像保存先のディレクトリ
save_dir = 'images'

# ディレクトリが存在しない場合は作成
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 透過PNG画像の合成関数
def overlay_image_alpha(img, overlay, pos, scale=1.0):
    overlay = cv2.resize(overlay, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    x, y = pos
    h, w = overlay.shape[0], overlay.shape[1]

    # 境界チェック（オーバーレイ画像が元の画像をはみ出ないようにする）
    if x + w > img.shape[1]:
        w = img.shape[1] - x
    if y + h > img.shape[0]:
        h = img.shape[0] - y
    if x < 0:
        x = 0
    if y < 0:
        y = 0

    if overlay.shape[2] == 4:  # 透過チャンネルがある場合
        alpha_mask = overlay[:h, :w, 3] / 255.0
        alpha_image = 1.0 - alpha_mask

        # オーバーレイ部分を合成
        for c in range(0, 3):
            img[y:y+h, x:x+w, c] = (alpha_image * img[y:y+h, x:x+w, c] + alpha_mask * overlay[:h, :w, c])

    return img

# 口の開閉を判定する関数
def calc_mouth(mouth):
    A = distance.euclidean(mouth[2], mouth[10])
    B = distance.euclidean(mouth[4], mouth[8])
    C = distance.euclidean(mouth[0], mouth[6])
    MAR = (A + B) / (2.0 * C)
    return round(MAR, 3)

# カウントダウンを表示しながらフレームを更新する関数
def start_countdown(cap, detector, face_parts_detector):
    start_time = time.time()  # カウントダウンの開始時間
    while True:
        ret, frame = cap.read()  # フレームを再取得
        if not ret:
            break

        # カウントダウンタイマー計算
        elapsed_time = time.time() - start_time
        countdown_time = COUNTDOWN_SECONDS - int(elapsed_time)

        # カウントダウンが終了したらループを抜ける
        if countdown_time <= 0:
            break

        # グレースケールに変換
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 顔検出
        rects = detector(gray, 1)

        for rect in rects:
            # ランドマークを取得
            shape = face_parts_detector(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # クマさんフィルターを適用
            frame = apply_bear_filter(frame, shape)

        # カウントダウンの数字を表示
        cv2.putText(frame, str(countdown_time), (frame.shape[1] // 2 - 50, frame.shape[0] // 2), 
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5, cv2.LINE_AA)
        
        # フレームを表示
        cv2.imshow('Preview', frame)

        # 'q'キーでカウントダウン中断
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    return frame  # カウントダウン後の最後のフレームを返す

# クマさんフィルターを適用する関数
def apply_bear_filter(image, shape):
    left_eye = shape[36]
    right_eye = shape[45]
    nose = shape[30]
    chin = shape[8]

    face_width = np.linalg.norm(left_eye - right_eye)
    face_height = np.linalg.norm(nose - chin)

    # 左耳と右耳の位置計算
    left_ear_x = int(left_eye[0]) - int(face_width * 0.5)
    left_ear_y = int(left_eye[1]) - int(face_height * 1.2)
    right_ear_x = int(right_eye[0]) + int(face_width * -0.2)
    right_ear_y = int(right_eye[1]) - int(face_height * 1.25)

    # クマの耳と鼻を合成
    image = overlay_image_alpha(image, left_bear_ear, (left_ear_x, left_ear_y), scale=face_width / 200)
    image = overlay_image_alpha(image, right_bear_ear, (right_ear_x, right_ear_y), scale=face_width / 200)

    # クマの鼻を合成
    nose_x = nose[0] - 58
    nose_y = nose[1] - 70
    image = overlay_image_alpha(image, bear_nose, (nose_x, nose_y), scale=0.3)

    # 口のランドマークを取得し、口が開いているか確認
    mouth = shape[48:68]
    MAR = calc_mouth(mouth)
    if MAR > 0.6:
        mouth_center = mouth.mean(axis=0).astype(int)
        lip_distance = distance.euclidean(mouth[3], mouth[9])
        image = overlay_image_alpha(image, cookie_img, (mouth_center[0] - 20, mouth_center[1] - int(lip_distance * 0.5)), scale=0.5)

    return image

# 画像を保存する関数
def save_image(frame):
    filename = f'photo_{time.strftime("%Y%m%d_%H%M%S")}.png'
    filepath = os.path.join(save_dir, filename)
    cv2.imwrite(filepath, frame)
    print(f"画像が保存されました: {filepath}")

# カメラデバイスをオープン
cap = cv2.VideoCapture(0)

###################
# メイン処理       #
###################
while True:
    ret, image = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)

    for rect in rects:
        shape = face_parts_detector(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # クマさんフィルターをリアルタイムで適用
        image = apply_bear_filter(image, shape)

    # フレームを表示
    cv2.imshow('Preview', image)

    # 's'キーでカウントダウン開始
    if cv2.waitKey(1) & 0xFF == ord('s'):
        # カウントダウンを開始し、終了時のフレームを取得
        final_frame = start_countdown(cap, detector, face_parts_detector)

        # カウントダウン後のフレームにクマさんフィルターを適用
        gray = cv2.cvtColor(final_frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 1)

        for rect in rects:
            shape = face_parts_detector(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # カウントダウン後のフレームにクマさんフィルターを適用
            final_frame = apply_bear_filter(final_frame, shape)

        # フィルター適用済みの画像を保存
        save_image(final_frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


画像が保存されました: images\photo_20241003_160231.png
画像が保存されました: images\photo_20241003_160251.png
